In [238]:
from mip import Model, xsum, minimize, BINARY, OptimizationStatus
from random import randint

In [424]:
class OptimizeModelWarehouse():
    def __init__(self, cap, cost, d, coord_ware, coord_client):
        self.cap = cap
        self.cost = cost
        self.d = d
        
        self.N = len(cap)
        self.M = len(d)
        
        self.distances = [[self._calculate_euclidean_distance(crd_clnt, crd_wr) for crd_wr in coord_ware] for crd_clnt in coord_client]
        
        self.status = None
        self.model = None
        self.a = None
        
    def optimize(self, max_time: int = 3600):
        self.model = Model()

        self.a = [[self.model.add_var(var_type=BINARY) for _ in range(self.N)] for _ in range(self.M)]

        self.model.objective = minimize(xsum((1 if sum([x[j] for x in self.a]) != 0 else 0) * self.cost[j] for j in range(self.N)) +
                               xsum(xsum(self.a[i][j] * self.distances[i][j] for i in range(self.M)) for j in range(self.N)))

        for i in range(self.M):
            self.model += xsum(self.a[i][j] for j in range(self.N)) == 1

        for j in range(self.N):
            self.model += xsum(self.a[i][j] * self.d[i] for i in range(self.M)) <= self.cap[j]
        
        self.status = status = self.model.optimize(max_seconds=3600)
        
    def print_result(self):
        obj, clients = self._find_obj_clients()
        opt = self._find_opt()
        
        if opt == 0:
            print("There is no optimal solution.")
            return

        print(' '.join(map(str, [obj, opt])))
        
        for i, client in enumerate(clients):
            print(f"{i} warehouse used by: {' '.join(map(str, client))}")
    
    def _find_opt(self):
        if self.status == OptimizationStatus.OPTIMAL:
            opt = 1
        else:
            opt = 0
        
        return opt
    
    def _find_obj_clients(self):
        clients = []
        obj = 0
        
        for j in range(self.N):
            inter = []
            
            for i in range(self.M):
                if self.a[i][j].x == 1:
                    if len(inter) == 0:
                        obj += self.cost[j]
                    inter.append(i)
            clients.append(inter)

        return obj, clients

    def _calculate_euclidean_distance(self, f_coord: list, s_coord: list):
        """
        Calculate Euclidean distance only for 2(two) dimensions.

        params f_coord: list[int, int]: coordinates(x, y) of first object.
        params s_coord: list[int, int]: coordinates(x, y) of second object.
        return int: return Euclidean distance. 
        """

        if len(f_coord) != 2 or len(s_coord) != 2:
            raise ValueError("Only 2(two) dimensions distance can be calculated.")

        return ((f_coord[0] - s_coord[0]) ** 2 + (f_coord[1] - s_coord[1]) ** 2) ** (1/2)


### Example of class usage.

In [425]:
def generate_distances(n: int, min_range: int = 0, max_range: int = 100):
    """
    Generate pair of [x, y] where x and y coordinates.
    
    params n: int: number pairs.
    params min_range: int; default = 0: minimum range of surface.
    params max_range: int; default = 100: maximum range of surface.
    return [lists]: list of pairs.
    """
    
    distances = []
    for _ in range(n):
        x = randint(min_range, max_range)
        y = randint(min_range, max_range)
        distances.append([x, y])
    
    return distances
    

In [432]:
cap = [230, 130, 180, 310, 170, 105]
cost = [1100, 1500, 2000, 3500, 1000, 3030]
d = [20, 30, 50, 55, 33, 90, 25, 60, 66, 42, 19, 24, 97, 11, 55, 7, 113, 84, 49, 77]

coord_ware = generate_distances(len(cap))
coord_client = generate_distances(len(d))


In [433]:
model = OptimizeModelWarehouse(cap, cost, d, coord_ware, coord_client)
model.optimize()
model.print_result()

12130 1
0 warehouse used by: 1 3 4 17
1 warehouse used by: 9 19
2 warehouse used by: 5 8 13
3 warehouse used by: 7 14 16 18
4 warehouse used by: 0 6 10 12 15
5 warehouse used by: 2 11


### Example of usage from txt file.

In [444]:
files = ['fl_1000_2', 'fl_100_14', 'fl_100_8', 'fl_200_2', 'fl_25_1',
         'fl_500_6', 'fl_50_6', 'fl_100_2', 'fl_100_9', 'fl_200_3', 'fl_1000_3',
         'fl_25_2', 'fl_500_7', 'fl_100_3', 'fl_16_1', 'fl_200_4', 'fl_25_3',
         'fl_50_1', 'fl_100_4', 'fl_16_2', 'fl_200_5', 'fl_25_4', 'fl_50_2',
         'fl_100_5', 'fl_2000_2', 'fl_200_6', 'fl_25_5', 'fl_50_3',
         'fl_100_6', 'fl_2000_3', 'fl_200_7', 'fl_3_1', 'fl_50_4',
         'fl_100_7', 'fl_200_1', 'fl_200_8', 'fl_4000_1', 'fl_50_5',
         'fl_100_11', 'fl_100_1', 'fl_100_10', 'fl_100_12', 'fl_100_13']

In [456]:
filename = 'data/' + 'fl_25_1'

with open(filename, 'r') as f:
    N, M = map(int, f.readline().strip().split())
    cap, cost = [], []
    d = []
    coord_ware, coord_client = [], []

    for _ in range(N):
        cap_, cost_, coord_ware_x, coord_ware_y = map(float, f.readline().strip().split())
        cap.append(cap_)
        cost.append(cost_)
        coord_ware.append([coord_ware_x, coord_ware_y])

    for _ in range(M):
        d_, coord_client_x, coord_client_y = map(float, f.readline().strip().split())
        d.append(d_)
        coord_client.append([coord_client_x, coord_client_y])

In [457]:
model = OptimizeModelWarehouse(cap, cost, d, coord_ware, coord_client)
model.optimize()
model.print_result()

105000.0 1
0 warehouse used by: 
1 warehouse used by: 
2 warehouse used by: 
3 warehouse used by: 10
4 warehouse used by: 
5 warehouse used by: 
6 warehouse used by: 6 17 37
7 warehouse used by: 0 1 2 3 4 5 7 8 9 11 12 13 14 15 16 18 19 20 21 22 23 24 25 27 28 29 30 31 32 34 35 38 39 40 41 42 43 44 45 46 47 49
8 warehouse used by: 
9 warehouse used by: 48
10 warehouse used by: 
11 warehouse used by: 36
12 warehouse used by: 
13 warehouse used by: 
14 warehouse used by: 
15 warehouse used by: 
16 warehouse used by: 33
17 warehouse used by: 
18 warehouse used by: 
19 warehouse used by: 
20 warehouse used by: 
21 warehouse used by: 26
22 warehouse used by: 
23 warehouse used by: 
24 warehouse used by: 
